# Convolutional Neural Network 

## Learning by implementation

* **flow**

1. Reading dataset (TFRecord)
2. Building CNN
    > ReLU
   
   * Convolution layer 
       $$h^n = ReLU(W^T_{n-1} * h^{n-1}) + b$$
   * CNN에서 바뀐 것 
     - matmul > Convolution
     - sigmoid activation > ReLU activation (for improve performance)
   
3. Applying Dropout layer
  > 매 train step 마다 랜덤하게 정해준 비율 만큼의 노드를 제외 

In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 

/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Parser for TFRecord
def parser(serialized_example):
    features = {
        'age' : tf.FixedLenFeature([1], tf.int64),
        'img' : tf.FixedLenFeature([61*49], tf.int64)
    }
    
    parsed_feature = tf.parse_single_example(serialized_example, features)
    age = parsed_feature['age']
    img = parsed_feature['img']
    return age, img 

In [3]:
tf.reset_default_graph()

face_train = "./cnn_dataset/face_train.tfrecord"
face_test = "./cnn_dataset/face_test.tfrecord"

# Dataset
train_dataset = tf.data.TFRecordDataset(face_train).map(parser)
train_dataset = train_dataset.batch(32).shuffle(777)
test_dataset = tf.data.TFRecordDataset(face_test).map(parser)
test_dataset = test_dataset.batch(32).shuffle(777)


#새로운 Iterator
# Basically, dtype and shape have to be share with same value to both train and test datasets
itr = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
age, img = itr.get_next()

img = tf.reshape(img, [-1, 61, 49, 1]) # 차원 주의 !
img = tf.cast(img, tf.float32)

age = tf.reshape(age, [-1]) # 차원 주의 ! 
age_ohe = tf.one_hot(age, depth=3, axis=-1)

train_init_op = itr.make_initializer(train_dataset)
test_init_op = itr.make_initializer(test_dataset)

In [4]:
def model(x, activation, dropout_prob, reuse=False):
    # 10개의 feature map, 3x3 필터사이즈 
    conv1 = tf.layers.conv2d(img, filters=16, kernel_size=3, 
                             padding='SAME', activation=activation, 
                             reuse=reuse, name='conv1')
    # 2x2 maxpooling stride=2 
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2)
    
    conv2 = tf.layers.conv2d(pool1, filters=16, kernel_size=3,
                            padding="SAME", activation=activation,
                            reuse=reuse, name='conv2')
    pool2 = tf.layers.max_pooling2d(conv2, pool_size=2, strides=2)
    
    conv3 = tf.layers.conv2d(pool2, filters=32, kernel_size=3, 
                            padding="SAME", activation=activation,
                            reuse=reuse, name='conv3')
    pool3 = tf.layers.max_pooling2d(conv3, pool_size=3, strides=2)
    # flattening for going through 
    flat_size = int(pool3.shape[1]) * int(pool3.shape[2]) * int(pool3.shape[3])
    flat = tf.reshape(pool3, [-1, flat_size])
    # # or
    # flat = tf.layers.flatten(pool3)
    
    dropout1 = tf.layers.dropout(flat, rate=dropout_prob)
    fc1 = tf.layers.dense(dropout1, units=1000, reuse=reuse, 
                          name='fc1')

    dropout2 = tf.layers.dropout(fc1, rate=dropout_prob)
    out = tf.layers.dense(dropout2, 3, reuse=reuse, name='out')
    
    return out 

train_out = model(img, tf.nn.relu, 0.7)
test_out = model(img, tf.nn.relu, 1, True)

# 여기서도 loss 에서 소프트맥스를 구해준다.
loss = tf.losses.softmax_cross_entropy(age_ohe, train_out)
train_op = tf.train.GradientDescentOptimizer(1e-4).minimize=(loss)
summary = tf.summary.scalar('train_loss', loss)

pred = tf.argmax(tf.nn.softmax(test_out), axis=1)
accuracy = tf.metrics.accuracy(age, pred)


# prediction 
train_pred = tf.nn.softmax(train_out)
test_pred = tf.nn.softmax(test_out)
test_pred = tf.argmax(test_pred, axis=1)

for var in tf.trainable_variables():
    tf.summary.histogram('var {}'.format(var.name), var)
    
merged = tf.summary.merge_all()

saver = tf.train.Saver()

INFO:tensorflow:Summary name var conv1/kernel:0 is illegal; using var_conv1/kernel_0 instead.
INFO:tensorflow:Summary name var conv1/bias:0 is illegal; using var_conv1/bias_0 instead.
INFO:tensorflow:Summary name var conv2/kernel:0 is illegal; using var_conv2/kernel_0 instead.
INFO:tensorflow:Summary name var conv2/bias:0 is illegal; using var_conv2/bias_0 instead.
INFO:tensorflow:Summary name var conv3/kernel:0 is illegal; using var_conv3/kernel_0 instead.
INFO:tensorflow:Summary name var conv3/bias:0 is illegal; using var_conv3/bias_0 instead.
INFO:tensorflow:Summary name var fc1/kernel:0 is illegal; using var_fc1/kernel_0 instead.
INFO:tensorflow:Summary name var fc1/bias:0 is illegal; using var_fc1/bias_0 instead.
INFO:tensorflow:Summary name var out/kernel:0 is illegal; using var_out/kernel_0 instead.
INFO:tensorflow:Summary name var out/bias:0 is illegal; using var_out/bias_0 instead.


In [5]:
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    writer = tf.summary.FileWriter('cnn_dataset/', sess.graph)
    loss_pdf = []
    
    for epoch in range(10):
        sess.run(train_init_op)
        for train_step in range(9999999):
            try :
                _, _loss, _summ = sess.run([train_op, loss, summary])
                loss_pdf.append(_loss)
                writer.add_summary(_summ, train_step)
                    
            except tf.errors.OutOfRangeError :
                break
        sess.run(test_init_op)
        for test_step in range(99999999):
                try:
                    _acc = sess.run(accuracy)
                    
                except tf.errors.OutOfRangeError:
                    break
        print('epochs : {}, train_loss : {}, acc : {}'.format(epoch, _loss, _acc[0]))
        saver.save(sess, "./cnn_dataset/cnn")

epochs : 0, train_loss : 17.21306610107422, acc : 0.3211192488670349
epochs : 1, train_loss : 13.23499584197998, acc : 0.3180619776248932
epochs : 2, train_loss : 19.477638244628906, acc : 0.3148675262928009
epochs : 3, train_loss : 41.07842254638672, acc : 0.3173770606517792
epochs : 4, train_loss : 42.93489456176758, acc : 0.31743744015693665
epochs : 5, train_loss : 26.506134033203125, acc : 0.31693214178085327
epochs : 6, train_loss : 34.25714874267578, acc : 0.3174128532409668
epochs : 7, train_loss : 17.400615692138672, acc : 0.3167102634906769
epochs : 8, train_loss : 32.453163146972656, acc : 0.3169633150100708
epochs : 9, train_loss : 39.122859954833984, acc : 0.3172309994697571
